In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE146114"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE146114"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE146114.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE146114.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE146114.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Combining imaging- and gene-based hypoxia biomarkers in cervical cancer improves prediction of treatment failure independent of intratumor heterogeneity"
!Series_summary	"Emerging biomarkers based on medical images and molecular characterization of tumor biopsies open up for combining the two disciplines and exploiting their synergy in treatment planning. We compared pretreatment classification of cervical cancer patients by two previously validated imaging- and gene-based hypoxia biomarkers, evaluated the influence of intratumor heterogeneity, and investigated the benefit of combining them in prediction of treatment failure. The imaging-based biomarker was hypoxic fraction, determined from diagnostic dynamic contrast enhanced (DCE)-MR images. The gene-based biomarker was a hypoxia gene expression signature determined from tumor biopsies. Paired data were available for 118 patients. Intratumor heterogeneity was assessed by variance analysis of MR 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional, Dict, Any, Callable
import json

# 1. Gene Expression Data Availability
# From the background information, we know this is a gene expression dataset
# The background mentions Illumina WG-6 v3 and Illumina HT-12 v4 which are gene expression platforms
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Cervical Cancer)
# The key value 1 contains gene-based hypoxia status which is a binary trait related to cervical cancer hypoxia
trait_row = 1

# For age
# There is no age information in the sample characteristics
age_row = None

# For gender
# All patients are female (cervical cancer patients), but gender is not explicitly mentioned
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on hypoxia status
    if 'less hypoxic' in value.lower():
        return 0  # Less hypoxic
    elif 'more hypoxic' in value.lower():
        return 1  # More hypoxic
    else:
        return None  # Unknown or invalid value

def convert_age(value):
    """Convert age value to continuous"""
    # No age data available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # No gender data available
    return None

# 3. Save Metadata
# Initial validation - check if both gene and trait data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a dataframe from the sample characteristics dictionary provided in the previous step
    # The sample characteristics dictionary contains the relevant clinical information
    # Sample characteristics were provided in the previous step output
    sample_char_dict = {0: ['histology: Squamous cell carcinoma', 'histology: Adenosquamous'], 
                        1: ['gene-based hypoxia: More hypoxic', 'gene-based hypoxia: Less hypoxic'], 
                        2: ['imaging-based hypoxia: Less hypoxic', 'imaging-based hypoxia: More hypoxic'], 
                        3: ['combined hypoxia biomarker: More hypoxic', 'combined hypoxia biomarker: Less hypoxic'], 
                        4: ['cohort (gse36562 study): DCE-MRI cohort', 'tissue: cervical tumor', 'cohort (gse36562 study): Validation cohort', 'figo stage: 2B'], 
                        5: ['hypoxia score: High', 'hypoxia score: Low', 'figo stage: 2B', 'figo stage: 3B', 'rna isolation method: Trizol'], 
                        6: ['tissue: cervical tumor', 'cohort (gse36562 study): DCE-MRI cohort', 'biopsy: Pooled biopsies'], 
                        7: ['figo stage: 2B', 'figo stage: 3B', 'hypoxia score: Low', 'figo stage: 4A', 'figo stage: 3A', 'figo stage: 1B1', 'figo stage: 2A', 'figo stage: 1B2', np.nan], 
                        8: ['cohort: Validation cohort', 'cohort: basic cohort', np.nan], 
                        9: ['cohort (gse38964 study): Integrative cohort', 'lymph node status (gse38433 study): 1', np.nan, 'lymph node status (gse38433 study): 0', 'cohort (gse38964 study): Validation cohort'], 
                        10: ['3p status: Loss', 'cohort (gse38964 study): Integrative cohort', np.nan, '3p status: No loss', 'cohort (gse38964 study): Validation cohort', '3p status: Not determined'], 
                        11: [np.nan, '3p status: Loss', '3p status: No loss', '3p status: Not determined']}
    
    # Create a dataframe with the sample characteristics dictionary
    # The index will be the row numbers, and columns will be sample IDs (we'll create dummy IDs)
    max_samples = max(len(features) for features in sample_char_dict.values())
    sample_ids = [f"Sample_{i+1}" for i in range(max_samples)]
    
    clinical_data = pd.DataFrame(index=sample_char_dict.keys(), columns=sample_ids)
    
    # Fill the dataframe with the sample characteristics
    for row, features in sample_char_dict.items():
        for col, feature in enumerate(features):
            if col < len(sample_ids):
                clinical_data.loc[row, sample_ids[col]] = feature
    
    # Use geo_select_clinical_features to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the selected clinical features to the output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Sample_1': [1.0], 'Sample_2': [0.0], 'Sample_3': [nan], 'Sample_4': [nan], 'Sample_5': [nan], 'Sample_6': [nan], 'Sample_7': [nan], 'Sample_8': [nan], 'Sample_9': [nan]}
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE146114.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs (ILMN_*), not human gene symbols. 
# They need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, 441782.0, 55818.0, nan, nan], 'GI': [23525203.0, 89042416.0, 46358420.0, 7376124.0, 5437312.0], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation store identifiers and gene symbols
# After examining the preview, we can see that:
# 'ID' column contains the Illumina probe IDs (ILMN_*) matching the gene_data index
# 'Symbol' column contains the gene symbols we need to map to

# 2. Get gene mapping dataframe using the two relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# 3. Apply gene mapping to convert probe measurements to gene expression data
# This handles the many-to-many relationship between probes and genes
# If a probe maps to multiple genes, its expression is divided equally among those genes
# Then values for each gene are summed across all contributing probes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the new gene expression dataframe
print(f"Mapped gene expression data shape: {gene_data.shape}")
# Print the first few gene symbols to confirm successful mapping
print("First few gene symbols:")
print(gene_data.index[:5])


Mapped gene expression data shape: (18838, 58)
First few gene symbols:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Re-extract clinical data directly from the source file to get proper sample IDs
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Define the conversion function for the trait (gene-based hypoxia)
def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on hypoxia status
    if 'less hypoxic' in value.lower():
        return 0  # Less hypoxic
    elif 'more hypoxic' in value.lower():
        return 1  # More hypoxic
    else:
        return None  # Unknown or invalid value

# Extract clinical features with proper sample IDs
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Row 1 contains gene-based hypoxia information
    convert_trait=convert_trait
)

# Save the correctly formatted clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Print sample IDs from both datasets for diagnostic purposes
print("Gene data samples:", normalized_gene_data.columns[:5], "...")
print("Clinical data samples:", selected_clinical_df.columns[:5], "...")

# Find common samples between the datasets
common_samples = set(normalized_gene_data.columns).intersection(set(selected_clinical_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) > 0:
    # Filter both datasets to only include common samples
    normalized_gene_data = normalized_gene_data[list(common_samples)]
    selected_clinical_df = selected_clinical_df[list(common_samples)]
    
    # Now link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print("Linked data shape:", linked_data.shape)
    
    # Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print("Linked data shape after handling missing values:", linked_data.shape)
    
    if linked_data.shape[0] > 0:
        # 4. Determine whether the trait and some demographic features are severely biased
        is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        # 5. Conduct quality check and save the cohort information
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=unbiased_linked_data,
            note="Study on hypoxia biomarkers in cervical cancer patients. Trait is gene-based hypoxic status."
        )
        
        # 6. If the linked data is usable, save it as a CSV file
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            unbiased_linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable and was not saved")
    else:
        print("No samples remained after handling missing values. Dataset is unusable.")
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=True,  # We consider empty datasets as biased
            df=pd.DataFrame(),
            note="All samples were filtered out due to missing trait values or excessive missing genes."
        )
else:
    print("No common samples between clinical and gene expression data. Cannot create linked dataset.")
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # We consider datasets with no overlap as biased
        df=pd.DataFrame(),
        note="No overlapping samples between clinical and gene expression data."
    )

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE146114.csv
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE146114.csv
Gene data samples: Index(['GSM1868975', 'GSM1868976', 'GSM1868977', 'GSM1868978', 'GSM1868979'], dtype='object') ...
Clinical data samples: Index(['GSM1868975', 'GSM1868976', 'GSM1868977', 'GSM1868978', 'GSM1868979'], dtype='object') ...
Number of common samples: 58
Linked data shape: (58, 17552)


Linked data shape after handling missing values: (58, 17552)
For the feature 'Cervical_Cancer', the least common label is '1.0' with 13 occurrences. This represents 22.41% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cervical_Cancer/GSE146114.csv
